In [117]:
#pip install elasticsearch

# Módulos

In [118]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime

# Inputs 

In [119]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-05 00:00:00"
fecha_fin_incidencia    = "2022-03-06 00:00:00"
# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"

In [120]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Nuevas Variables

In [121]:
# Transformacion de variables de entrada

# Fechas
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia = datetime.timestamp(fecha_inicio_incidencia)

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia = datetime.timestamp(fecha_fin_incidencia)

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

# Cliente de ML
cliente_ml = client.ml

# Solución

In [122]:
# Paso 1. Parar y cerrar el Job
cliente_ml.close_job(job_id=jobname)

ObjectApiResponse({'closed': True})

In [123]:
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
snapshots_available
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))

for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # fecha_inicio_incidencia_timestamp = timestamp("2022-04-07T11:22:16Z")
  fecha_inicio_incidencia_timestamp = 1.65209341e+12
  # Ordenar por timestamp (primera columna)
  latest1 = latest[latest[:,0].argsort()]
  latest2 = np.flip(latest1, axis=0)
  # Eliminar valores mayores
  latest3 = latest2[latest2[:,0]<fecha_inicio_incidencia_timestamp]
# Escoger el id de la reversion correcta
id_revert = latest[0,1]

In [124]:
# Paso 3. Revertir el Job antes de la incidencia 
cliente_ml.revert_model_snapshot(job_id=jobname, snapshot_id=int(id_revert), delete_intervening_results=True)

ObjectApiResponse({'model': {'job_id': 'sabado_anomalia', 'min_version': '8.3.0', 'timestamp': 1669311586000, 'description': 'State persisted due to job close at 2022-11-24T17:39:46+0000', 'snapshot_id': '1669311586', 'snapshot_doc_count': 1, 'model_size_stats': {'job_id': 'sabado_anomalia', 'result_type': 'model_size_stats', 'model_bytes': 77448, 'peak_model_bytes': 78164, 'model_bytes_exceeded': 0, 'model_bytes_memory_limit': 11534336, 'total_by_field_count': 3, 'total_over_field_count': 0, 'total_partition_field_count': 2, 'bucket_allocation_failures_count': 0, 'memory_status': 'ok', 'assignment_memory_basis': 'current_model_bytes', 'categorized_doc_count': 0, 'total_category_count': 0, 'frequent_category_count': 0, 'rare_category_count': 0, 'dead_category_count': 0, 'failed_category_count': 0, 'categorization_status': 'ok', 'log_time': 1669311586859, 'timestamp': 1669310400000}, 'latest_record_time_stamp': 1669311000000, 'latest_result_time_stamp': 1669310400000, 'retain': False}})

In [125]:
# Paso 4. Abrir el Job 
cliente_ml.open_job(job_id=jobname)

ObjectApiResponse({'opened': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

In [126]:
# Paso 5. Avanzamos datafeed hasta la fecha de inicio de incidencia
cliente_ml.start_datafeed(datafeed_id=datafeed_id, end=str(int(fecha_inicio_incidencia)))

ObjectApiResponse({'started': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

In [128]:
# Paso 6. Abrir el Job 
cliente_ml.open_job(job_id=jobname)

ObjectApiResponse({'opened': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

In [129]:
# Paso 7. Avanzamos datafeed desde la fecha de fin de incidencia hasta la fecha actual
cliente_ml.start_datafeed(datafeed_id=datafeed_id, start=str(int(fecha_fin_incidencia)))

ObjectApiResponse({'started': True, 'node': '1UbOBFWrRuiIttmnruod0A'})